# S3準拠オブジェクトストレージを大容量データの保管場所として使う

このNotebookは[AWS S3](https://aws.amazon.com/jp/s3/)に準拠したオブジェクトストレージを大容量データの保管場所として利用する際のサポートを提供します。

## ０．オブジェクトストレージとの接続設定をする

データ保管の場所として、オブジェクトストレージをシステムが認識するための設定を行います。

※ここでは、適切なキーとシークレットアクセスキーとのIDが必要です。詳しくは、各オブジェクトストレージのドキュメントをご覧いただくか、各管理者にお問い合わせください。

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = input('アクセスキーID: ')
os.environ['AWS_SECRET_ACCESS_KEY'] = input('シークレットアクセスキー: ')

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def on_click_callback(clicked_button: widgets.Button) -> str:
    with output:
        output.clear_output()
        # セレクタで選択されているアイテムを使う
        print(f'{select.value}が選択されました。')
        print('次のセルでは、このオブジェクトストレージへの接続処理を進めます。')
        print('選択したストレージでよろしければ、次のセルの実行へお進みください。')
        print('もし選び直す場合は、もう一度上のボックスから選択した後、「決定」ボタンを押してください。')
        
print('どのオブジェクトストレージを使いますか？')
print('\t※AWS S3: Amazon Web Service上で稼働するストレージ\n\t  mdx S3: mdx環境上で稼働するストレージ')

list_s3 = ['AWS S3', 'mdx S3']

# select.valueの値を次のセルで処理の場合分けに利用する
select = widgets.Select(options=list_s3)
button = widgets.Button(description='決　定')
button.on_click(on_click_callback)

output = widgets.Output(layour={'border': '1px solid black'})

display(select, button, output)

In [ ]:
# Note: 前のセルから、select.valueを利用している

import time

# 次のセルでも利用する変数のため、関数の外で宣言している
special_remote_name = 's3remote'

def connect_s3_storage():
    bucket_name = special_remote_name + str(time.time())

    try:
        if select.value == 'AWS S3':
            # for Amazon Web Service(AWS) S3
            !git-annex initremote $special_remote_name type=S3 bucket=$bucket_name encryption=none autoenable=true
        elif select.value == 'mdx S3':
            # for mdx(s3ds.mdx.jp) S3
            !git-annex initremote $special_remote_name type=S3 port=443 host=s3ds.mdx.jp bucket=$bucket_name encryption=none signature=v2 requeststyle=path autoenable=true
        else:
            raise ValueError('select.value is wrong: ' + select.value) 
        print(select.value + 'への接続設定が完了しました。')
    except:
        print('処理を完了できませんでした。\n利用するオブジェクトストレージが選択されていない可能性があります。')
        print('前のセルを実行いただき、利用したいオブジェクトストレージを選んでください。')
        print('もし選択してもこのメッセージが表示されている場合、恐れ入りますがデータガバナンス機能ヘルプセンター（仮）へお問い合わせください。')
        return

if __name__ == '__main__':
    connect_s3_storage()

## 1-A. 実データをオブジェクトストレージへ保存する

この手順では、「０．オブジェクトストレージとの接続設定をする」で設定したオブジェクトストレージへ、実データをアップロードする処理のサポートを提供します。

（ここに改めてgit-annexのデータの扱い方の前提を書く）

In [ ]:
# Note: 前のセルで宣言した変数special_remote_nameを利用している
# level 0: git-annex管理下のすべての実データをアップロード
import papermill as pm

# Git-annex管理するパスの配列を作成する
annexed_save_path = ['input_data', 'output_data']

%cd ~/WORKFLOW/FLOW/util
pm.execute_notebook(
    './base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SIBLING_NAME=special_remote_name, SAVE_MESSAGE='[GIN] データを保存', PATH=annexed_save_path, TO_GIT=False)
)
print('データ同期が完了しました。')

## 1-B. 実データをオブジェクトストレージから取得する

この手順では、「０．オブジェクトストレージとの接続設定をする」で設定したオブジェクトストレージから、実データをダウンロードする処理のサポートを提供します。

In [ ]:
# datalad get?